In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops

# Fungsi untuk menentukan label berdasarkan folder tempat gambar berada
def get_label_from_folder(folder_name):
    if 'utuh' in folder_name.lower():
        return 'utuh'
    elif 'patah' in folder_name.lower():
        return 'patah'
    elif 'rusak' in folder_name.lower():
        return 'rusak'
    else:
        return None  # Kembalikan None jika tidak ada label yang sesuai

# Fungsi untuk ekstraksi fitur
def extract_features(image_path, label, jenis):
    try:
        # Membaca gambar
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Gambar {image_path} tidak dapat dibaca.")
    except Exception as e:
        print(f"Error pada {image_path}: {e}")
        return None  # Mengembalikan None jika ada error
    
    # Mengubah gambar ke grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Deteksi tepi menggunakan Canny
    edges = cv2.Canny(gray, 100, 200)

    # Mengubah gambar ke ruang warna HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Ekstraksi fitur warna (rata-rata nilai Hue, Saturation, dan Value)
    avg_hue = np.mean(hsv[:, :, 0])  # Rata-rata Hue
    avg_saturation = np.mean(hsv[:, :, 1])  # Rata-rata Saturation
    avg_value = np.mean(hsv[:, :, 2])  # Rata-rata Value
    
    # Ekstraksi fitur tekstur menggunakan GLCM
    glcm = graycomatrix(gray, [1], [0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    
    # Menghitung ukuran gambar (pixel)
    height, width = image.shape[:2]
    size = height * width
    
    # Menghitung bentuk menggunakan area kontur
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    area = 0
    for cnt in contours:
        area += cv2.contourArea(cnt)
    
    # Menyusun hasil ekstraksi fitur
    features = {
        'filename': os.path.basename(image_path),
        'avg_hue': avg_hue,  # Hue
        'avg_saturation': avg_saturation,  # Saturation
        'avg_value': avg_value,  # Value
        'contrast': contrast,
        'dissimilarity': dissimilarity,
        'homogeneity': homogeneity,
        'energy': energy,
        'correlation': correlation,
        'size': size,
        'area': area,
        'kind': jenis  # Fitur manual jenis gabah
        'quality': label,  # Kualitas gabah: utuh, patah, rusak
    }
    
    return features

# Menyimpan fitur ke dalam DataFrame dan CSV
def save_features_to_csv(input_folder, output_csv):
    data = []
    
    # Iterasi melalui subfolder (utuh, patah, rusak) di dalam folder input
    for folder_name in os.listdir(input_folder):
        folder_path = os.path.join(input_folder, folder_name)
        
        # Pastikan itu adalah folder dan bukan file
        if os.path.isdir(folder_path):
            # Tentukan label kualitas gabah berdasarkan nama folder
            label = get_label_from_folder(folder_name)
            if label is None:
                print(f"Folder {folder_name} tidak memiliki label yang sesuai, lewati.")
                continue
            
            # Input manual untuk 'jenis' gabah
            jenis = input(f"Masukkan jenis gabah untuk folder '{folder_name}': ")

            # Iterasi melalui file gambar dalam folder
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)
                
                # Memeriksa ekstensi file untuk memastikan itu adalah gambar
                if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    print(f"File {filename} bukan gambar, lewati.")
                    continue
                
                # Ekstrak fitur dari gambar
                features = extract_features(file_path, label, jenis)
                if features:  # Hanya menambahkan fitur yang valid
                    data.append(features)
                else:
                    print(f"Gambar {filename} gagal diproses.")
    
    # Membuat DataFrame dari data
    df = pd.DataFrame(data)
    
    # Menyimpan DataFrame ke CSV
    df.to_csv(output_csv, index=False)
    print(f"Data berhasil disimpan dalam {output_csv}")

# Direktori gambar (folder utama yang berisi subfolder utuh, patah, rusak)
input_folder = r'C:\Users\Naufal\OneDrive\MyDocuments\KULIAH\Tugas Kuliah\TUGAS AKHIR\gabah beras merah'  # Ganti dengan direktori gambar Anda
output_csv = 'gabah_beras merah_coba.csv'

# Menyimpan fitur ke dalam CSV
save_features_to_csv(input_folder, output_csv)

Masukkan jenis gabah untuk folder 'patah':  beras merah
Masukkan jenis gabah untuk folder 'rusak':  beras merah
Masukkan jenis gabah untuk folder 'utuh':  beras merah


Data berhasil disimpan dalam gabah_features_beras merah.csv
